# 01_replication_asm_v6

In [1]:
import pandas as pd
import numpy as np
import fs_qe as fq

%matplotlib inline

1. The database we use here is the relative well-studied Chinese Annual Survey of Manufacturing (ASM), an extensive yearly survey of Chinese manufacturing firms from 1998 to 2013.
   - Please find the links attached:
     - [access from PKU](https://www.lib.pku.edu.cn/portal/cn/news/0000001637)
   - The ASM is weighted towards medium and large firms, includes all Chinese manufacturing firms with total annual sales of more than 5 million RMB (approximately U.S.$\$ 800,000$), as well additional state-owned firms with lower sales.
   - This survey provides detailed information on ownership, location, production, and the balance sheet of manufacturing firms.

# 1 pre-process the data
We can skip the next 2 cells to read csv file directly.

In [2]:
%%time
df = pd.read_stata('/Users/shuhu/Desktop/1_papers_w:john/04_firm_size/github/firm_size_empirics/orbis/firm_growth_asset/data/asie_readytouse.dta')

CPU times: user 55.7 s, sys: 1min 1s, total: 1min 56s
Wall time: 2min 12s


In [ ]:
# convert the database from dta to csv
df.to_csv('/Users/shuhu/Desktop/1_papers_w:john/04_firm_size/github/firm_size_empirics/orbis/firm_growth_asset/data/asie_readytouse.csv')

##### Please start from the code below.

The ``asie_readytouse.dta`` database collects financially corporate data for all firms over total asset ¥5 million from 1998 to 2013 in the mainland China.
- Since the data in 2010 are fake, we delete all observations in 2010.
- The observations for an individual firm are not consecutive over years.

In [ ]:
df= pd.read_csv('/Users/shuhu/Desktop/1_papers_w:john/04_firm_size/github/firm_size_empirics/orbis/firm_growth_asset/data/asie_readytouse.csv')

In [ ]:
df = df[['asie_id', 'year', 'tasset']] # asie_id ascending denotes each individual firm
df1=df.astype({'asie_id':int, 'year': int}).set_index(['asie_id', 'year'])
dfx = df1.unstack().stack(dropna=False) # This treatment solves the missing-value issue

# 2 process the data

## 2.1 generating the firm-growth vs firm size database

### 2.1.1 the firm-growth as $\frac{S_+- S}{S_+}$

In [ ]:
df2 = dfx
df2 = df2[df2['tasset'] != 0.0]
df2['fgrow'] = df2.groupby('asie_id')['tasset'].pct_change(periods=1) * 100
df3 = df2.dropna()
df4 = df3[(df3.index != 1998) & (df3.index != 2011)]
df5 = df4.reset_index()
df5['logtasset'] = np.log(df5['tasset'])

In [ ]:
fq.scatterplot(df5, x='tasset', y='fgrow')

There are two missing-data issues in this data processing:
- Since we delete observations in 2010, the firm growth rates corresponding to firms in 2011 must be wrong.
  - we want to delete those observations in 2011.
  - **Solved!**
- Since for some firms, their observations are consecutive over years.
  - e.g., for the firm with ``asie_id == 1``, there are only 2011 & 2012 observations.
  - the firm growth rate we calculated might not be the annually firm growth rate.
  - Solution: consider filling in missing rows in their consecutive years.
    - For example, there is a possible way to solve this issue: https://stackoverflow.com/questions/40093971/pandas-dataframe-insert-fill-missing-rows-from-previous-dates
    
Also, we want to change the growth rates as $\ln S_{+} - \ln S$ and firm size as $S$.

### 2.1.2 the firm-growth as $\ln (S_+) - \ln (S) $ against firm size $S$ or log firm size $\ln S$

In [ ]:
df7 = dfx.reset_index('asie_id')
df7['logtasset'] = np.log(df7['tasset'])
df7['fgrow_log'] = df7.groupby('asie_id')['logtasset'].diff(periods=-1) * 100
df8 = df7.dropna()
df9 = df8[(df8.index != 1998) & (df8.index != 2011)]

df7

In [ ]:
fq.scatterplot(df9, y='fgrow_log')

In [ ]:
fq.scatterplot(df9, x='logtasset', y='fgrow_log', xlabel='Log Size (Log Total Assets)')

### 2.1.3 Ruling out some outliers at the lower bound of the firm size

In [ ]:
df10 = df9[df9['tasset'] > 10]
df10

In [ ]:
fq.scatterplot(df10, y='fgrow_log')

In [ ]:
fq.scatterplot(df10, x='logtasset',y='fgrow_log', xlabel='Log Size (Log Total Assets)')

## 2.2 generating double-log plots with a $45$-degree line

### 2.2.1 process

In [ ]:
df12 = fq.process45line(dfx, ID='asie_id')

In [ ]:
df13 = df12[(df12.index != 1998) & (df12.index != 2011)]
df13

### 2.2.2 Scatter Plots with Linear Regression Approximation (with 45 degree line)

In [ ]:
fq.plots(df13,reg=True)

In [ ]:
fq.plots(df13, x='logtasset_prev', y='logtasset', reg=True)

### 2.2.3 Scatter Plots with 3rd-Degree Polynomial Approximation (with 45 degree line)

In [ ]:
fq.plots(df13)